# IBM Applied Data Science Capstone

### Opening a New Shopping Mall in Kuala Lumpur, Malaysia

- Build a dataframe of neighborhoods in Kuala Lumpur, Malaysia by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall

In [1]:
#Importing libraries
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text

In [3]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [4]:
# create a list to store neighborhood data
neighborhoodList = []

# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    

# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [5]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [6]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [7]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df.head()

(71, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148494,101.696729
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610


In [8]:
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516964, 101.6942371.


In [9]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [10]:
# define Foursquare Credentials and Version
CLIENT_ID = 'BR4AVAJNS2XD3EM4LS4ZUQEFOMBN5GSF04TOOIDZUBQT0DGP' # your Foursquare ID
CLIENT_SECRET = '3XEZF24QGP3BO0AEF2KA4IOFOFA1LRZOSVYSX44HGE4WDANK' # your Foursquare Secret
VERSION = '20200628' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BR4AVAJNS2XD3EM4LS4ZUQEFOMBN5GSF04TOOIDZUBQT0DGP
CLIENT_SECRET:3XEZF24QGP3BO0AEF2KA4IOFOFA1LRZOSVYSX44HGE4WDANK


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
kualalumpur_venues = getNearbyVenues(names=kl_df['Neighborhood'],
                                   latitudes=kl_df['Latitude'],
                                   longitudes=kl_df['Longitude']
                                  )

Alam Damai
Ampang, Kuala Lumpur
Bandar Menjalara
Bandar Sri Permaisuri
Bandar Tasik Selatan
Bandar Tun Razak
Bangsar
Bangsar Park
Bangsar South
Batu 11 Cheras
Batu, Kuala Lumpur
Brickfields
Bukit Bintang
Bukit Jalil
Bukit Kiara
Bukit Nanas
Bukit Petaling
Bukit Tunku
Cheras, Kuala Lumpur
Chow Kit
Damansara Heights
Damansara Town Centre
Damansara, Kuala Lumpur
Dang Wangi
Desa Petaling
Federal Hill, Kuala Lumpur
Happy Garden
Jalan Cochrane, Kuala Lumpur
Jalan Duta
Jinjang
Kampung Baru, Kuala Lumpur
Kampung Datuk Keramat
Kampung Padang Balang
Kepong
Kepong Baru
KL Eco City
Kuchai Lama
Lembah Pantai
Maluri
Medan Tuanku
Miharja
Mont Kiara
Pantai Dalam
Pudu, Kuala Lumpur
Putrajaya
Salak South
Segambut
Semarak
Sentul, Kuala Lumpur
Setapak
Setiawangsa
Shamelin
Sri Hartamas
Sri Petaling
Sungai Besi
Taman Bukit Maluri
Taman Connaught
Taman Desa
Taman Ibukota
Taman Len Seng
Taman Melati
Taman Midah
Taman OUG
Taman P. Ramlee
Taman Sri Sinar
Taman Taynton View
Taman Tun Dr Ismail
Taman U-Thant
Taman

In [16]:
kualalumpur_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
1,Alam Damai,3.05769,101.74388,閒茶素食店 Leisure Tea Vegetarian,3.057673,101.747258,Vegetarian / Vegan Restaurant
2,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
3,Alam Damai,3.05769,101.74388,Roti Canai Perindu,3.055071,101.740574,Breakfast Spot
4,Alam Damai,3.05769,101.74388,Heng Kee Tim Sum 興記點心,3.057513,101.747191,Dim Sum Restaurant


In [17]:
kualalumpur_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alam Damai,19,19,19,19,19,19
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,51,51,51,51,51,51
Bandar Sri Permaisuri,40,40,40,40,40,40
Bandar Tasik Selatan,35,35,35,35,35,35
Bandar Tun Razak,13,13,13,13,13,13
Bangsar,28,28,28,28,28,28
Bangsar Park,28,28,28,28,28,28
Bangsar South,69,69,69,69,69,69


In [18]:
print('There are {} uniques categories.'.format(len(kualalumpur_venues['Venue Category'].unique())))

There are 281 uniques categories.


In [21]:
kualalumpur_venues['Venue Category'].unique()[:50]

array(['Noodle House', 'Vegetarian / Vegan Restaurant',
       'Chinese Restaurant', 'Breakfast Spot', 'Dim Sum Restaurant',
       'Dessert Shop', 'Farmers Market', 'Convenience Store',
       'Japanese Restaurant', 'Hakka Restaurant', 'Pharmacy',
       'Fast Food Restaurant', 'Food Court', 'Asian Restaurant',
       'Food Truck', 'Padangnese Restaurant', 'Hostel',
       'South Indian Restaurant', 'Exhibit', 'Chettinad Restaurant',
       'Indian Restaurant', 'Monument / Landmark', 'Park', 'Hotel',
       'Athletics & Sports', 'Restaurant', 'Gym', 'Café', 'Coffee Shop',
       'Juice Bar', 'Theater', 'Bakery', 'Bridal Shop', 'Flower Shop',
       'Art Gallery', 'Food Stand', 'Wine Bar', 'Flea Market', 'Bar',
       'Bed & Breakfast', 'Thai Restaurant', 'Filipino Restaurant',
       'Cupcake Shop', 'Malay Restaurant', 'Other Nightlife',
       'Music Venue', 'Sandwich Place', 'Bubble Tea Shop',
       'Clothing Store', 'Deli / Bodega'], dtype=object)

In [23]:
"Neighborhood" in kualalumpur_venues['Venue Category'].unique()

True

## Analysing Each Neighborhood

In [25]:
# one hot encoding
kl_onehot = pd.get_dummies(kualalumpur_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = kualalumpur_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(2670, 282)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport Food Court,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Belgian Restaurant,Betting Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Stadium,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Imperial Restaurant,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Laundromat,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Poke Place,Police Station,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Tunnel,Turkish Restaurant,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyb

In [26]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(71, 282)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,Airport Food Court,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Bank,Bar,Basketball Court,Bed & Breakfast,Beer Bar,Belgian Restaurant,Betting Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Stadium,Comedy Club,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Imperial Restaurant,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Laundromat,Lebanese Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nature Preserve,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoors & Recreation,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Perfume Shop,Pet Café,Pet Store,Pharmacy,Pizza Place,Platform,Playground,Plaza,Poke Place,Police Station,Pool,Pool Hall,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Road,Rock Climbing Spot,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,TV Station,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toll Plaza,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Tunnel,Turkish Restaurant,Udon Restaurant,Vacation Rental,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyb

In [27]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

13

In [28]:
#Creating new dataset for Shopping Mall only
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [29]:
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.0
1,"Ampang, Kuala Lumpur",0.0
2,Bandar Menjalara,0.0
3,Bandar Sri Permaisuri,0.0
4,Bandar Tasik Selatan,0.0


## Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [30]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int32)

In [31]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [32]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.0,0
1,"Ampang, Kuala Lumpur",0.0,0
2,Bandar Menjalara,0.0,0
3,Bandar Sri Permaisuri,0.0,0
4,Bandar Tasik Selatan,0.0,0


In [33]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.0,0,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.0,0,3.148494,101.696729
2,Bandar Menjalara,0.0,0,3.190350,101.625450
3,Bandar Sri Permaisuri,0.0,0,3.103910,101.712260
4,Bandar Tasik Selatan,0.0,0,3.072750,101.714610


In [34]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged.head()

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.0,0,3.057690,101.743880
36,Kuchai Lama,0.0,0,3.090740,101.677330
37,Lembah Pantai,0.0,0,3.121202,101.663899
38,Maluri,0.0,0,3.147890,101.694050
39,Medan Tuanku,0.0,0,3.159260,101.698340


In [36]:
import numpy as np
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  Examine Clusters

In [37]:
#Cluster 1
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.000000,0,3.057690,101.743880
36,Kuchai Lama,0.000000,0,3.090740,101.677330
37,Lembah Pantai,0.000000,0,3.121202,101.663899
38,Maluri,0.000000,0,3.147890,101.694050
39,Medan Tuanku,0.000000,0,3.159260,101.698340
40,Miharja,0.000000,0,3.147890,101.694050
42,Pantai Dalam,0.000000,0,3.094760,101.667470
43,"Pudu, Kuala Lumpur",0.000000,0,3.133540,101.713070
44,Putrajaya,0.000000,0,3.125843,101.718509
45,Salak South,0.000000,0,3.081020,101.697240


In [39]:
#Cluster 2
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
24,Desa Petaling,0.038462,1,3.083300,101.704380
25,"Federal Hill, Kuala Lumpur",0.027778,1,3.136370,101.685640
8,Bangsar South,0.028986,1,3.111020,101.662830
41,Mont Kiara,0.014925,1,3.165320,101.652430
27,"Jalan Cochrane, Kuala Lumpur",0.019608,1,3.132903,101.724679
21,Damansara Town Centre,0.037037,1,3.136442,101.690296
66,Taman Tun Dr Ismail,0.015385,1,3.152830,101.622710
30,KL Eco City,0.020000,1,3.117130,101.673840
70,Wangsa Maju,0.028571,1,3.203910,101.737190


In [40]:
#Cluster 3
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
17,Bukit Tunku,0.076923,2,3.17381,101.68276


## Observation
Most of the shopping malls are concentrated in the central area of Kuala Lumpur city, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 3 has very low number to totally no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. 

Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 3 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 2 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 1 which already have high concentration of shopping malls and suffering from intense competition.